In [476]:
# pip install openai==1.40.1

import openai
from openai import OpenAI

from pydantic import BaseModel

In [477]:
# put your openai key here
openai_key = '...'
model = 'gpt-4o-2024-08-06'

## This is where we collect all the intial information

In [478]:
client = OpenAI(api_key= openai_key)

# this initialises the chatbot memeory
history = ''

# this is the instruction that is persistently passed to the model 
system = """You are a helpful assistant with the only role to help a human create a good schedule for their business. \n
     You role is to guide the human through the following points so that they can generate a good schedule, step by step, one question at the time: \n
     1. What is the business name?\n
     2. What type of business is it?\n
     3. Where is the business located? In case of multiple branches in the same city, please provide a unique location identifier, such as the neighborrhood or street.\n
     4. What days do you want the business to be opened?\n
     5. What times do you want the business to be opened?\n
     6. Specify all the roles that the business need to have covered in order to function well (e.g., cook, receptionist, waiter)?\n
     7. Once all the above information has been provieded, output all the parameters you have collected in a json format, without any additional text, so that they can be rendered from a front-end"""

# this is the chat function, takes our system instruction, user message and chatbot memory
def chat(system, user, history):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": f'{history + user}'},
        ],
        # response_format=Schema,    # this is where you can define any schema for the output
    )

    message = completion.choices[0].message
    return message

In [508]:
# chat with the model here, just updating the "user" message as you go through the conversation
user = 'Hello there!'

# this shouldn't require touching, we just get the response and update the chatbot memory / conversation as we move through the conversation
response = chat(system, user, history)
history += (f'Human message: {user}\n')
history += (f'\nAI message: {response.content}\n')
print(history) 

Human message: Hello!

AI message: Hello! Let's get started on creating a good schedule for your business. What is the name of your business?
Human message: Alessandro Pizza, is going to be great!

AI message: Great! Alessandro Pizza sounds delicious. What type of business is it?
Human message: Pizzeria, an Italian Pizzeria!

AI message: What a fantastic choice! Now, where is Alessandro Pizza located? If there are multiple branches in the same city, please provide a unique location identifier, such as the neighborhood or street.
Human message: I only have one, in Central London!

AI message: Great! Alessandro Pizza is located in Central London. What days do you want the business to be opened?
Human message: Every day 8AM to 8PM

AI message: AI message: Great! So, you want Alessandro Pizza to be open every day. Now, just to confirm, what times do you want the business to be opened each day? You mentioned 8 AM to 8 PM, is that correct?
Human message: Yes!

AI message: AI message: Perfect

Problem: this last step, where we generate the summary, maybe could be handled by another bot specifically prompet to return a strcutured format?

## Here we create the schedule

In [305]:
scheduler_system = "You are a helpful assistant, based on the given information will provide a schedule where roles are assigned per each day and return it in a json format"

In [513]:
def scheduler(scheduler_system, user, history):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": scheduler_system},
            {"role": "user", "content": f'{history + user}'},
        ],
        # response_format=BusinessSchedule,    # this is where you can define any schema for the output
    )

    message = completion.choices[0].message
    return message

In [521]:
user = """Based on the below information that was collected during a conversation with our client, provide me with a schedule where roles are assigned per each day, making sure the business is well covered at all times.\n """

In [522]:
scheduler_response = scheduler(scheduler_system, user, history)

In [523]:
print(scheduler_response.content)

Based on the provided information, here's a suggested schedule for Alessandro Pizza in Central London. Since the pizzeria is open from 8 AM to 8 PM, we've divided the shifts to ensure coverage throughout the day. The schedule considers having each role filled for the entire duration of opening hours:

```json
{
  "Central London": {
    "Mon": {
      "Cook/Pizza Chef": "08:00-20:00",
      "Waiter/Waitress": "08:00-14:00, 14:00-20:00",
      "Cashier": "08:00-14:00, 14:00-20:00",
      "Cleaner": "11:00-13:00, 17:00-19:00"
    },
    "Tue": {
      "Cook/Pizza Chef": "08:00-20:00",
      "Waiter/Waitress": "08:00-14:00, 14:00-20:00",
      "Cashier": "08:00-14:00, 14:00-20:00",
      "Cleaner": "11:00-13:00, 17:00-19:00"
    },
    "Wed": {
      "Cook/Pizza Chef": "08:00-20:00",
      "Waiter/Waitress": "08:00-14:00, 14:00-20:00",
      "Cashier": "08:00-14:00, 14:00-20:00",
      "Cleaner": "11:00-13:00, 17:00-19:00"
    },
    "Thu": {
      "Cook/Pizza Chef": "08:00-20:00",
      

## If you want to start using schemas

In [524]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional

# this is how you can define pydantic schemas and pass them to the model (you will need a main class)

class DayRole(BaseModel):
    day: str = Field(..., description="Day of the week, e.g., Mon")
    role: str = Field(..., description="Role of the person, e.g., Cook")
    start_time: str = Field(..., description="Start time, e.g., 08:00")
    end_time: str = Field(..., description="End time, e.g., 16:00")

class Person(BaseModel):
    name: str = Field(..., description="Name of the person")
    working_hours: List[DayRole] = Field(..., description="List of roles and hours for each day")

class Location(BaseModel):
    location: str = Field(..., description="Location name, e.g., London")
    staff: List[Person] = Field(..., description="List of people working at this location")

class Schedule(BaseModel):
    locations: List[Person] = Field(..., description="List of locations and their schedules")


response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
            {"role": "system", "content": scheduler_system},
            {"role": "user", "content": f'{history + user}'},
    ],
    response_format= Schedule
)

print(response.choices[0].message.content)

{"locations":[{"name":"Central London","working_hours":[{"day":"Monday","role":"Cook/Pizza Chef","start_time":"08:00","end_time":"16:00"},{"day":"Monday","role":"Waiter/Waitress","start_time":"08:00","end_time":"16:00"},{"day":"Monday","role":"Cashier","start_time":"12:00","end_time":"20:00"},{"day":"Monday","role":"Cleaner","start_time":"16:00","end_time":"20:00"},{"day":"Tuesday","role":"Cook/Pizza Chef","start_time":"08:00","end_time":"16:00"},{"day":"Tuesday","role":"Waiter/Waitress","start_time":"08:00","end_time":"16:00"},{"day":"Tuesday","role":"Cashier","start_time":"12:00","end_time":"20:00"},{"day":"Tuesday","role":"Cleaner","start_time":"16:00","end_time":"20:00"},{"day":"Wednesday","role":"Cook/Pizza Chef","start_time":"08:00","end_time":"16:00"},{"day":"Wednesday","role":"Waiter/Waitress","start_time":"08:00","end_time":"16:00"},{"day":"Wednesday","role":"Cashier","start_time":"12:00","end_time":"20:00"},{"day":"Wednesday","role":"Cleaner","start_time":"16:00","end_time":"

In [525]:
import json

content = response.choices[0].message.content
# Parsing the JSON string into a Python dictionary
parsed_json = json.loads(content)
# Pretty-print the JSON with indentation
formatted_json = json.dumps(parsed_json, indent=4)
print(formatted_json)

{
    "locations": [
        {
            "name": "Central London",
            "working_hours": [
                {
                    "day": "Monday",
                    "role": "Cook/Pizza Chef",
                    "start_time": "08:00",
                    "end_time": "16:00"
                },
                {
                    "day": "Monday",
                    "role": "Waiter/Waitress",
                    "start_time": "08:00",
                    "end_time": "16:00"
                },
                {
                    "day": "Monday",
                    "role": "Cashier",
                    "start_time": "12:00",
                    "end_time": "20:00"
                },
                {
                    "day": "Monday",
                    "role": "Cleaner",
                    "start_time": "16:00",
                    "end_time": "20:00"
                },
                {
                    "day": "Tuesday",
                    "role": "Cook/Pizza Chef